In [5]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files


In [6]:
#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

In [7]:
#!conda install -c conda-forge folium=0.11.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # plotting library

print('Libraries imported.')

Libraries imported.


Defined Problem: I'm planning to visit Torontonext summer and want to check which area is best food and also good to stay.

In [73]:
#set up Foursquare credentials
ClIENT_ID = 'TDAVIOVNZNTNV2PDXTHBDQCNGL3W2TIVKNY5ZBMB4E5EMTXS' # your Foursquare ID
ClIENT_SECRET = 'IP2HVJSJNCC55QU35Q5YLQ3G5FZCQPCICI22EFEIMLPTI4HA' # your Foursquare Secret
VERSION = '20200726'
LIMIT =100
radius=1000
print('Your credentails:')
print('Foursquare_ID: ' + ClIENT_ID)
print('Foursquare_Secret:' + ClIENT_SECRET)

Your credentails:
Foursquare_ID: TDAVIOVNZNTNV2PDXTHBDQCNGL3W2TIVKNY5ZBMB4E5EMTXS
Foursquare_Secret:IP2HVJSJNCC55QU35Q5YLQ3G5FZCQPCICI22EFEIMLPTI4HA


In [74]:
# get Denver's latitude & longitude 
address = 'Toronto'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
lat = location.latitude
long = location.longitude
print(latitude, longitude)

43.6534817 -79.3839347


In [75]:
#search resturants in Denver
search_query= "Restaurant"
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(ClIENT_ID,ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=TDAVIOVNZNTNV2PDXTHBDQCNGL3W2TIVKNY5ZBMB4E5EMTXS&client_secret=IP2HVJSJNCC55QU35Q5YLQ3G5FZCQPCICI22EFEIMLPTI4HA&ll=43.6534817,-79.3839347&v=20200726&query=Restaurant&radius=1000&limit=100'

In [91]:
#search hotels in Denver
search_query2= "Hotel"
ur2 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(ClIENT_ID,ClIENT_SECRET, latitude, longitude, VERSION, search_query2
                                                                                                                                   , radius, LIMIT)
ur2

'https://api.foursquare.com/v2/venues/search?client_id=TDAVIOVNZNTNV2PDXTHBDQCNGL3W2TIVKNY5ZBMB4E5EMTXS&client_secret=IP2HVJSJNCC55QU35Q5YLQ3G5FZCQPCICI22EFEIMLPTI4HA&ll=43.6534817,-79.3839347&v=20200726&query=Hotel&radius=1000&limit=100'

In [92]:
#Send the GET Request and examine the results
results = requests.get(url).json()
results2 = requests.get(ur2).json()

In [98]:
# assign relevant part of JSON to venues
venues = results['response']['venues']
dataframe = json_normalize(venues)
dataframe.head()

venues2 = results2['response']['venues']
dataframe2 = json_normalize(venues2)

In [99]:
#format table

# keep only columns that include venue name, and anything that is associated with location for resturants
clean_col = ['name', 'categories'] + [col for col in dataframe2.columns if col.startswith('location.')]+ ['id']
clean_df = dataframe.loc[:,clean_columns]
clean_df.head()

,name,categories,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,id
0,Hemispheres Restaurant & Bistro,"[{'id': '4bf58dd8d48988d14e941735', 'name': 'A...",110 Chestnut Street,CA,Toronto,Canada,NaN,224,"[110 Chestnut Street, Toronto ON M5G 1R3, Canada]","[{'label': 'display', 'lat': 43.65488413420439...",43.654884,-79.385931,NaN,M5G 1R3,ON,4ad4c05ff964a52048f720e3
1,The Hot House Restaurant & Bar,"[{'id': '4bf58dd8d48988d14e941735', 'name': 'A...",35 Church St,CA,Toronto,Canada,at Front St E,973,"[35 Church St (at Front St E), Toronto ON M5E ...","[{'label': 'display', 'lat': 43.64882370529773...",43.648824,-79.373702,NaN,M5E 1T3,ON,4ada5d5bf964a520e92121e3
2,Victoria's Restaurant,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",37 King Street East,CA,Toronto,Canada,at Le Meridien King Edward Hotel,763,[37 King Street East (at Le Meridien King Edwa...,"[{'label': 'display', 'lat': 43.64929834396347...",43.649298,-79.376431,NaN,M5C 1E9,ON,4ad4c05cf964a52006f620e3
3,Aroma Fine Indian Restaurant,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",287 King St. W,CA,Toronto,Canada,at John St.,906,"[287 King St. W (at John St.), Toronto ON M5V ...","[{'label': 'display', 'lat': 43.64646252150344...",43.646463,-79.389644,NaN,M5V 1J5,ON,4aef8854f964a5201cd921e3
4,Some Time BBQ Grill Restaurant 碳烤屋,"[{'id': '52af3b773cf9994f4e043c03', 'name': 'S...",988 Baldwin Street,CA,Toronto,Canada,NaN,839,"[988 Baldwin Street, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.655874, 'lng':...",43.655874,-79.393826,NaN,NaN,ON,5750b013498e755287c6de97


In [100]:
# keep only columns that include venue name, and anything that is associated with location for hotels
clean_col2 = ['name', 'categories'] + [col for col in dataframe2.columns if col.startswith('location.')]+ ['id']
clean_df2 = dataframe2.loc[:,clean_columns]
clean_df2.head()

,name,categories,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,id
0,Sheraton Centre Toronto Hotel,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",123 Queen Street West,CA,Toronto,Canada,at York St.,324,"[123 Queen Street West (at York St.), Toronto ...","[{'label': 'display', 'lat': 43.6505944, 'lng'...",43.650594,-79.384530,NaN,M5H 2M9,ON,4ab2d511f964a5209b6c20e3
1,Pantages Hotel & Spa,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",200 Victoria St,CA,Toronto,Canada,at Shuter St,410,"[200 Victoria St (at Shuter St), Toronto ON, C...","[{'label': 'display', 'lat': 43.65449797039222...",43.654498,-79.379035,NaN,NaN,ON,4ae61cf6f964a520caa421e3
2,Chelsea Hotel,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",33 Gerrard Street West,CA,Toronto,Canada,at Yonge St,562,"[33 Gerrard Street West (at Yonge St), Toronto...","[{'label': 'display', 'lat': 43.65849759157591...",43.658498,-79.383097,NaN,M5G 1Z4,ON,51d212c3498ebf27dc469bc9
3,One King West Hotel & Residence,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",1 King St W,CA,Toronto,Canada,at Yonge St.,686,"[1 King St W (at Yonge St.), Toronto ON M5H 1A...","[{'label': 'display', 'lat': 43.6491395, 'lng'...",43.649139,-79.377876,NaN,M5H 1A1,ON,4af96fbbf964a520c01122e3
4,The Rex Hotel Jazz & Blues Bar,"[{'id': '4bf58dd8d48988d1e7931735', 'name': 'J...",194 Queen St W,CA,Toronto,Canada,Queen & St. Patrick,499,"[194 Queen St W (Queen & St. Patrick), Toronto...","[{'label': 'display', 'lat': 43.65050475544005...",43.650505,-79.388577,NaN,M5V 1Z1,ON,4b68aed1f964a520de862be3


In [102]:
# delete unnecessary columns
#resturants
clean_dataframe= clean_df.drop(['location.cc', 'location.city', 'location.country', 'location.crossStreet','location.distance','location.labeledLatLngs', 'location.state',
                                       'id'], axis=1)

#hotel
clean_dataframe2= clean_df2.drop(['location.cc', 'location.city', 'location.country', 'location.crossStreet','location.distance','location.labeledLatLngs', 'location.state',
                                       'id'], axis=1)


In [105]:
# create dataframe of resturants, hotels
df = pd.concat([clean_dataframe, clean_dataframe2], ignore_index=True)
df.head()

,name,categories,location.address,location.formattedAddress,location.lat,location.lng,location.neighborhood,location.postalCode
0,Hemispheres Restaurant & Bistro,"[{'id': '4bf58dd8d48988d14e941735', 'name': 'A...",110 Chestnut Street,"[110 Chestnut Street, Toronto ON M5G 1R3, Canada]",43.654884,-79.385931,NaN,M5G 1R3
1,The Hot House Restaurant & Bar,"[{'id': '4bf58dd8d48988d14e941735', 'name': 'A...",35 Church St,"[35 Church St (at Front St E), Toronto ON M5E ...",43.648824,-79.373702,NaN,M5E 1T3
2,Victoria's Restaurant,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",37 King Street East,[37 King Street East (at Le Meridien King Edwa...,43.649298,-79.376431,NaN,M5C 1E9
3,Aroma Fine Indian Restaurant,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",287 King St. W,"[287 King St. W (at John St.), Toronto ON M5V ...",43.646463,-79.389644,NaN,M5V 1J5
4,Some Time BBQ Grill Restaurant 碳烤屋,"[{'id': '52af3b773cf9994f4e043c03', 'name': 'S...",988 Baldwin Street,"[988 Baldwin Street, Toronto ON, Canada]",43.655874,-79.393826,NaN,NaN


In [111]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list4 = row['categories']
    except:
        categories_list4 = row['venue.categories']
        
    if len(categories_list4) == 0:
        return None
    else:
        return categories_list4[0]['name']
# filter the category for each row
df['categories'] = df.apply(get_category_type, axis=1) 
df.head()

,name,categories,location.address,location.formattedAddress,location.lat,location.lng,location.neighborhood,location.postalCode
0,Hemispheres Restaurant & Bistro,American Restaurant,110 Chestnut Street,"[110 Chestnut Street, Toronto ON M5G 1R3, Canada]",43.654884,-79.385931,NaN,M5G 1R3
1,The Hot House Restaurant & Bar,American Restaurant,35 Church St,"[35 Church St (at Front St E), Toronto ON M5E ...",43.648824,-79.373702,NaN,M5E 1T3
2,Victoria's Restaurant,Restaurant,37 King Street East,[37 King Street East (at Le Meridien King Edwa...,43.649298,-79.376431,NaN,M5C 1E9
3,Aroma Fine Indian Restaurant,Indian Restaurant,287 King St. W,"[287 King St. W (at John St.), Toronto ON M5V ...",43.646463,-79.389644,NaN,M5V 1J5
4,Some Time BBQ Grill Restaurant 碳烤屋,Szechuan Restaurant,988 Baldwin Street,"[988 Baldwin Street, Toronto ON, Canada]",43.655874,-79.393826,NaN,NaN


In [117]:
#visualize resturant and hotels in toronoto 
map_resturant_hotel = folium.Map(location=[latitude, longitude], zoom_start=15)

# add markers to map
for lat, lng, name, address in zip(df['location.lat'], df['location.lng'], df['name'], df['location.address']):
    label = '{}, {}'.format(name, location.address)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_resturant_hotel)  
    
map_resturant_hotel